# Simple Amazon Bedrock Flow Operations

Basic operations with Bedrock Flow APIs without any wrapper functions.

In [ ]:
!pip install boto3

In [ ]:
import boto3
import json

# Initialize AWS clients
session = boto3.Session(profile_name='default', region_name='us-east-1')
bedrock_client = session.client('bedrock-agent')
bedrock_runtime = session.client('bedrock-agent-runtime')
iam = session.client('iam')

In [ ]:
!aws sts get-caller-identity

## 1. Create IAM Role

In [ ]:
# Create or get IAM role
role_name = 'BedrockFlowsRole'
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "bedrock.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

bedrock_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
        "Sid": "BedrockFlowPermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:CreateFlow",
            "bedrock:UpdateFlow",
            "bedrock:GetFlow",
            "bedrock:ListFlows", 
            "bedrock:DeleteFlow",
            "bedrock:ValidateFlowDefinition", 
            "bedrock:CreateFlowVersion",
            "bedrock:GetFlowVersion",
            "bedrock:ListFlowVersions",
            "bedrock:DeleteFlowVersion",
            "bedrock:CreateFlowAlias",
            "bedrock:UpdateFlowAlias",
            "bedrock:GetFlowAlias",
            "bedrock:ListFlowAliases",
            "bedrock:DeleteFlowAlias",
            "bedrock:InvokeFlow",
            "bedrock:TagResource",
            "bedrock:UntagResource", 
            "bedrock:ListTagsForResource"
        ],
        "Resource": "*"
        },
        {
        "Sid": "BedrockResourcePermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:ApplyGuardrail",
            "bedrock:InvokeGuardrail",
            "bedrock:InvokeModel",
            "bedrock:GetCustomModel",
            "bedrock:InvokeAgent",
            "bedrock:Retrieve",
            "bedrock:RetrieveAndGenerate",
            "bedrock:GetPrompt",
            "bedrock:ListPrompts",
            "bedrock:RenderPrompt"
        ],
        "Resource": "*"
        },
        {
            "Sid": "GetBedrockResources",
            "Effect": "Allow",
            "Action": [
                "bedrock:GetAgent",
                "bedrock:GetKnowledgeBase",
                "bedrock:GetGuardrail",
                "bedrock:GetPrompt",
            ],
            "Resource": "*"
        }
    ]
}

try:
    # Try to get existing role
    role_response = iam.get_role(RoleName=role_name)
    role_arn = role_response['Role']['Arn']
    print(f"Using existing role: {role_arn}")
    
except iam.exceptions.NoSuchEntityException:
    # Role doesn't exist, create it
    print(f"Creating new role: {role_name}")
    role_response = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy)
    )
    role_arn = role_response['Role']['Arn']
    
    # Attach Bedrock policy
    policy_name = 'BedrockFlowsPolicy'
    iam.put_role_policy(
        RoleName=role_name,
        PolicyName=policy_name,
        PolicyDocument=json.dumps(bedrock_policy)
    )
    
    print(f"Created role: {role_arn}")
    
except Exception as e:
    print(f"Error handling IAM role: {str(e)}")
    raise e

## 2. Create Flow

In [ ]:
# Simple flow definition
flow_definition = {
  "nodes": [
    {
      "type": "Input",
      "name": "FlowInputNode",
      "outputs": [{ "name": "document", "type": "Array" }]
    },
    {
      "type": "Iterator",
      "name": "PromptIterator",
      "inputs": [{ "name": "array", "type": "Array", "expression": "$.data" }],
      "outputs": [
        { "name": "arrayItem", "type": "String" },
        { "name": "arraySize", "type": "Number" }
      ]
    },
    {
      "type": "Prompt",
      "name": "PromptProcessor",
      "configuration": {
        "prompt": {
          "sourceConfiguration": {
            "inline": {
              "modelId": "anthropic.claude-3-5-sonnet-20240620-v1:0",
              "templateType": "TEXT",
              "inferenceConfiguration": {
                "text": { "temperature": 0.7, "topP": 1 }
              },
              "templateConfiguration": { "text": { "text": "{{userPrompt}}" } }
            }
          }
        }
      },
      "inputs": [
        { "name": "userPrompt", "type": "String", "expression": "$.data" }
      ],
      "outputs": [{ "name": "modelCompletion", "type": "String" }]
    },
    {
      "type": "Collector",
      "name": "ResponseCollector",
      "inputs": [
        { "name": "arrayItem", "type": "String", "expression": "$.data" },
        { "name": "arraySize", "type": "Number", "expression": "$.data" }
      ],
      "outputs": [{ "name": "collectedArray", "type": "Array" }]
    },
    {
      "type": "Output",
      "name": "FlowOutput",
      "inputs": [
        { "name": "document", "type": "Array", "expression": "$.data" }
      ]
    }
  ],
  "connections": [
    {
      "name": "Input_to_Iterator",
      "source": "FlowInputNode",
      "target": "PromptIterator",
      "type": "Data",
      "configuration": {
        "data": { "sourceOutput": "document", "targetInput": "array" }
      }
    },
    {
      "name": "Iterator_to_Prompt",
      "source": "PromptIterator",
      "target": "PromptProcessor",
      "type": "Data",
      "configuration": {
        "data": { "sourceOutput": "arrayItem", "targetInput": "userPrompt" }
      }
    },
    {
      "name": "Prompt_to_Collector",
      "source": "PromptProcessor",
      "target": "ResponseCollector",
      "type": "Data",
      "configuration": {
        "data": {
          "sourceOutput": "modelCompletion",
          "targetInput": "arrayItem"
        }
      }
    },
    {
      "name": "Iterator_to_Collector_Size",
      "source": "PromptIterator",
      "target": "ResponseCollector",
      "type": "Data",
      "configuration": {
        "data": { "sourceOutput": "arraySize", "targetInput": "arraySize" }
      }
    },
    {
      "name": "Collector_to_Output",
      "source": "ResponseCollector",
      "target": "FlowOutput",
      "type": "Data",
      "configuration": {
        "data": { "sourceOutput": "collectedArray", "targetInput": "document" }
      }
    }
  ]
}

# Create flow
create_response = bedrock_client.create_flow(
    name='SimpleFlow1',
    description='A simple flow example',
    definition=flow_definition,
    executionRoleArn=role_arn
)

flow_id = create_response['id']
print(f"Created flow: {flow_id}")

## 3. Prepare Flow

In [ ]:
# Prepare flow
bedrock_client.prepare_flow(flowIdentifier=flow_id)

# Create version
version_response = bedrock_client.create_flow_version(flowIdentifier=flow_id)
flow_version = version_response['version']

# Create alias
alias_response = bedrock_client.create_flow_alias(
    flowIdentifier=flow_id,
    name='latest',
    description='Latest version',
    routingConfiguration=[{'flowVersion': flow_version}]
)

alias_id = alias_response['id']
print(f"Flow version: {flow_version}")
print(f"Alias ID: {alias_id}")

## 4. Invoke Flow

In [ ]:
# Invoke flow
input_payload = {
    "content": {"document": ["Hello, Flow!"]},
    "nodeName": "FlowInputNode",
    "nodeOutputName": "document"
}

response = bedrock_runtime.invoke_flow(
    flowIdentifier=flow_id,
    flowAliasIdentifier=alias_id,
    inputs=[input_payload]
)

# Process response stream
for event in response.get("responseStream", []):
    print(json.dumps(event, indent=2))

## 5. List Flows

In [ ]:
# List all flows
flows = bedrock_client.list_flows()
for flow in flows['flowSummaries']:
    print(f"Flow ID: {flow['id']}")
    print(f"Name: {flow['name']}")
    if 'description' in flow:
        print(f"Description: {flow['description']}")
    print("-" * 50)

## 6. Delete Flow

In [ ]:
# Delete flow resources in order
# notify user of flow alias, version and id being deleted
print (f"Deleting flow: {flow_id} with alias: {alias_id} and version: {flow_version}")
print (f"Please wait...")
# 1. Delete alias
bedrock_client.delete_flow_alias(
    flowIdentifier=flow_id,
    aliasIdentifier=alias_id
)

# 2. Delete version
bedrock_client.delete_flow_version(
    flowIdentifier=flow_id,
    flowVersion=flow_version
)

# 3. Delete flow
bedrock_client.delete_flow(flowIdentifier=flow_id)

print("Flow and associated resources deleted")